In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 16.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [7]:
def fetch_data_costco(search_term):
    options=webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver=webdriver.Chrome(options=options)

    driver.get("https://www.costco.com.tw/")

    search_input=WebDriverWait(driver,20).until(
        EC.presence_of_element_located((By.XPATH,"//*[@id='searchBoxContainer']/div/form/input"))
    )

    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)

    data=[]
    for page in range(1,4):
      if page>1:
        current_url=driver.current_url
        if "&page=" in current_url:
                new_url = current_url.split("&page=")[0] + f"&page={page}"
        else:
                new_url = current_url + f"&page={page}"

        print(f"Attempting to navigate to: {new_url}") #debug print

        driver.get(new_url)


      product_name=WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/main/div[4]/sip-product-listing/div[2]/div/sip-product-search-results/sip-page-slot/sip-product-list/div/section/div[3]/div/ul/sip-product-list-item/li/div[2]/div[3]/div/a[1]/span"))
    )
      prices=WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/main/div[4]/sip-product-listing/div[2]/div/sip-product-search-results/sip-page-slot/sip-product-list/div/section/div[3]/div/ul/sip-product-list-item/li/div[2]/div[2]/sip-product-price-panel/div/div[1]/span/sip-format-price/span"))
    )

      product_link=WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, "/html/body/main/div[4]/sip-product-listing/div[2]/div/sip-product-search-results/sip-page-slot/sip-product-list/div/section/div[3]/div/ul/sip-product-list-item/li/div[2]/div[3]/div/a[2]"))
    )

      for product,price,link in zip(product_name,prices,product_link):
          product_links = link.get_attribute("href")
          data.append({
          'Product':product.text,
          'Price':price.text,
          'Link':product_links
         })

    df=pd.DataFrame(data)
    driver.quit()
    return df


In [4]:
def fetch_data_pchome(search_term):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.get("https://24h.pchome.com.tw/")
    time.sleep(3)

    search_input = driver.find_element(By.CLASS_NAME, "c-search__input")
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "c-prodInfoV2__title"))
    )

    print("Redirected to:", driver.current_url)

    all_data = []

    for page in range(1, 6):  # Loop through 5 pages
        if page > 1:
            # Construct the URL for subsequent pages
            current_url = driver.current_url
            if "&page=" in current_url:
                new_url = current_url.split("&page=")[0] + f"&page={page}"
            else:
                new_url = current_url + f"&page={page}"

            print(f"Attempting to navigate to: {new_url}") #debug print

            driver.get(new_url)

            try:
                # Check for a unique element on each page (e.g., the first product title)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "c-prodInfoV2__title"))
                )
                first_title = driver.find_element(By.CLASS_NAME, "c-prodInfoV2__title").text
                print(f"Successfully navigated to page {page}. First product title: {first_title[:50]}...") #debug print
            except Exception as e:
                print(f"Failed to navigate to page {page} or page did not load correctly. Error: {e}")
                break #stop if page navigation failed

            time.sleep(3) #give time for items to load

        product_names = driver.find_elements(By.CLASS_NAME, "c-prodInfoV2__title")
        product_prices = driver.find_elements(By.CLASS_NAME, "c-prodInfoV2__priceValue--m")
        product_links = driver.find_elements(By.CSS_SELECTOR, "a.c-prodInfoV2__link.gtmClickV2")

        data = []

        for name, price, link in zip(product_names, product_prices, product_links):
            product_link = link.get_attribute("href")
            data.append({
                'Product': name.text,
                'Price': price.text,
                'Link': product_link
            })
        all_data.extend(data)

    df = pd.DataFrame(all_data)
    driver.quit()

    return df

In [5]:
item="iphone 16"
df1=fetch_data_pchome(item)
display(df1)



Redirected to: https://24h.pchome.com.tw/search/?q=iphone%2016
Attempting to navigate to: https://24h.pchome.com.tw/search/?q=iphone%2016&page=2
Successfully navigated to page 2. First product title: Apple 蘋果 iPhone 16e (256G)...
Attempting to navigate to: https://24h.pchome.com.tw/search/?q=iphone%2016&page=3
Successfully navigated to page 3. First product title: Apple 蘋果 iPhone 16e (256G)...
Attempting to navigate to: https://24h.pchome.com.tw/search/?q=iphone%2016&page=4
Successfully navigated to page 4. First product title: Apple 蘋果 iPhone 16e (256G)...
Attempting to navigate to: https://24h.pchome.com.tw/search/?q=iphone%2016&page=5
Successfully navigated to page 5. First product title: Apple 蘋果 iPhone 16e (256G)...


,Product,Price,Link
0,Apple 蘋果 iPhone 16e (256G),"$24,400",https://24h.pchome.com.tw/prod/DYAJ81-1900IDJHY
1,Apple 蘋果 iPhone 16e (128G),"$20,900",https://24h.pchome.com.tw/prod/DYAJ8L-1900IDJG6
2,Apple 蘋果 iPhone 16 Pro Max (256G),"$43,500",https://24h.pchome.com.tw/prod/DYAJMC-1900I4G81
3,Apple 蘋果 iPhone 16 Pro (512G),"$47,000",https://24h.pchome.com.tw/prod/DYAJM4-1900HSPQQ
4,Apple 蘋果 iPhone 16 Pro Max (256G),"$43,500",https://24h.pchome.com.tw/prod/DYAJMC-A900HSYEF
...,...,...,...
195,Apple 蘋果 iPhone 16 (256G),"$1,837",https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSB
196,Apple 蘋果 iPhone 16 Pro (256G),"$38,700",https://24h.pchome.com.tw/prod/DYAJM7-A900HSYDS
197,Apple 蘋果 iPhone 16 (256G),"$31,500",https://24h.pchome.com.tw/prod/DYAJLY-1900I4AHH
198,Apple 蘋果 iPhone 16 Pro Max 256GB 沙漠色鈦金屬,"$39,080",https://24h.pchome.com.tw/prod/DYAJMM-A900I5TTP


In [8]:
item="iphone 16"
df2=fetch_data_costco(item)
display(df2)

Attempting to navigate to: https://www.costco.com.tw/search?searchOption=taiwan%2Ftw-search-all&text=iphone%2016&page=2
Attempting to navigate to: https://www.costco.com.tw/search?searchOption=taiwan%2Ftw-search-all&text=iphone%2016&page=3


,Product,Price,Link
0,Apple iPhone 16e 128GB,"$20,899",https://www.costco.com.tw/Digital-Mobile/Mobil...
1,Apple iPhone 16e 256GB,"$24,399",https://www.costco.com.tw/Digital-Mobile/Mobil...
2,Apple iPhone 16e 512GB,"$31,299",https://www.costco.com.tw/Digital-Mobile/Mobil...
3,Apple iPhone 16 128GB,"$28,289",https://www.costco.com.tw/Digital-Mobile/Mobil...
4,Apple iPhone 16 256GB,"$32,059",https://www.costco.com.tw/Digital-Mobile/Mobil...
5,Apple iPhone 16 Plus 128GB,"$31,559",https://www.costco.com.tw/Digital-Mobile/Mobil...
6,Apple iPhone 16 Pro 128GB,"$34,989",https://www.costco.com.tw/Digital-Mobile/Mobil...
7,Apple iPhone 16 Pro 256GB,"$38,759",https://www.costco.com.tw/Digital-Mobile/Mobil...
8,Apple EarPods 具備 Lightning 連接器 (MWTY3FE/A),$539,https://www.costco.com.tw/Digital-Mobile/Weara...
9,Beats Pill 可攜式藍牙無線防潑抗水揚聲器,"$3,999",https://www.costco.com.tw/Digital-Mobile/Weara...


In [9]:
def sort_price(df):
  df['Price'] = df['Price'].replace({'\$': '', ',': ''}, regex=True)
  df['Price']=pd.to_numeric(df['Price'],errors='coerce')
  df=df.sort_values("Price")
  return df

In [11]:
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df=sort_price(merged_df)
merged_df=merged_df.drop_duplicates('Link')
keywords = item.lower().split()

pattern = ''.join(f'(?=.*{kw})' for kw in keywords)

merged_df = merged_df[merged_df['Product'].str.lower().str.contains(pattern, regex=True, na=False)]


merged_df.to_csv(f"Result.csv",index=False,encoding='utf-8') # note that in pc home allow installment so the price might seem so low
display(merged_df)


,Product,Price,Link
26,Apple 蘋果 iPhone 16 (128G),1644,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSR
75,Apple 蘋果 iPhone 16 (256G),1837,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSB
112,Apple 蘋果 iPhone 16 Plus (256G),2002,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSF
182,Apple 蘋果 iPhone 16 Pro (128G),2029,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSY
144,Apple 蘋果 iPhone 16 Pro Max (256G),2469,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTT4
23,Apple 蘋果 iPhone 16 Pro Max (512G),2854,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTT5
200,Apple iPhone 16e 128GB,20899,https://www.costco.com.tw/Digital-Mobile/Mobil...
1,Apple 蘋果 iPhone 16e (128G),20900,https://24h.pchome.com.tw/prod/DYAJ8L-1900IDJG6
201,Apple iPhone 16e 256GB,24399,https://www.costco.com.tw/Digital-Mobile/Mobil...
40,Apple 蘋果 iPhone 16e (256G),24400,https://24h.pchome.com.tw/prod/DYAJ81-1900IDJHY
